# Training and Testing A Visual Transformer Model
Here we test a visual Transfrom ViT from [An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale](https://arxiv.org/abs/2010.11929)

In [1]:
import torch
import torchvision as tv

USE_GPU = True
if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
#get models
print(f"Using {device} device")

Using cuda device


## Load our Dataset and Create our dataloaders

In [2]:
batch_size = 64
num_classes = 80

## Load our model

In [3]:
#hyperparameters:
lr = 0.001
#maximum learning rate we will let our model train in order to train faster at the start
max_lr = 0.01
weight_decay = 0.00001
EPOCHS = 100
#end hyperparameters
weights = tv.models.ViT_B_16_Weights.DEFAULT
transform = weights.transforms()
#model and optimizers
model = tv.models.vit_b_16(weights=weights)

loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
#scales the gradients, neccessary for mixed precision data types to properly converge
scaler = torch.amp.GradScaler(device=device)
#change our learning rate based on far we are in training and if we are improving
lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=max_lr, total_steps=EPOCHS) #*len(train_loader))

#added data to our model for ease of use (and to prevent passing so many variables to our training function)
model.device = device
model.name = "ViT_b_16"
model.path = "Trained_Models\\ViT\\" #where to save our best model
print(model.path)
#redfine our output layer to output our classes
model.heads.head = torch.nn.Linear(in_features=model.heads.head.in_features,out_features=num_classes,bias=True)
print(model)

Trained_Models\ViT\
VisionTransformer(
  (conv_proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  (encoder): Encoder(
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (encoder_layer_0): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (dropout): Dropout(p=0.0, inplace=False)
        (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=3072, out_features=768, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (encoder_layer_1): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=T

## Train our Model

In [4]:
#we will call the function we defined in "Training_Functions.py"
from Training_Functions import TrainModel

TrainModel(model,EPOCHS, loss_fn, train_loader, val_loader, optimizer, lr_scheduler, scaler)

NameError: name 'train_loader' is not defined

## Test our Model

In [ ]:
#we will call the function we defined in "Training_Functions.py"
from Training_Functions import TestModel
TestModel(model, test_loader, loss_fn)